In [8]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# Common imports
import numpy as np
import os
import pandas as pd
import seaborn as sns
import sklearn as sk
import matplotlib as mpl
import matplotlib.pyplot as plt

# to make this notebook's output stable across runs
np.random.seed(0)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

pd.set_option('display.max_columns', None)

from IPython.display import display, HTML

In [9]:
# Get current directory of file (so filesystem should work regardless of user machine)
cwd = os.getcwd()
print(cwd)
# Read in data from sources
play_by_play_path = os.path.join(cwd, 'Data')
play_by_play_path = os.path.join(play_by_play_path, 'syracuse_gsw_basic_pbp.csv')
play_by_play_df = pd.read_csv(play_by_play_path)
results_path = os.path.join(cwd, 'Data')
results_path = os.path.join(results_path, "syracuse_gsw_basic_results.csv")
game_results_df = pd.read_csv(results_path)



C:\Users\perso\Documents\GitHub\syracusebasketballchallenge


In [10]:
# Reset indices
play_by_play_df.set_index('id', inplace= True)
game_results_df.set_index('id', inplace= True)
display(play_by_play_df.head(2))

,season,season_type,nba_game_id,home,away,game_date,quarter,poss_id,off_team,def_team,poss_time,oncourt_id,play_type,off_def,player_type,player_name,player_nba_id,play_clock,sequence_id,play_id,result_id,seq_result_id,play_zone,last_updated_time
id,,,,,,,,,,,,,,,,,,,,,,,,
1219,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,1,LAL,GSW,720,1,Rim Gather,OFF,Ball-Handler,DeAndre Jordan,201599.0,716,1,2,0022100002_1_1,0022100002_1_1,Rim,26:31.2
1227,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,3,LAL,GSW,694,1,Initiation,OFF,Ball-Handler,Russell Westbrook,201566.0,689,3,6,NaN,0022100002_1_3,Mid-Left Above Break 3,26:31.2


In [11]:
## Add didscore boolean

game_results_df['did_score'] = (game_results_df['points'] > 0).astype(int)

In [12]:
## Filter only for plays when the warriors were on offense

gsw_off_plays_df = play_by_play_df.loc[(play_by_play_df["off_team"] == "GSW") & (play_by_play_df["off_def"] == "OFF")].copy()

## Jettison unlabeled data - no result
gsw_off_plays_df = gsw_off_plays_df.loc[gsw_off_plays_df["seq_result_id"].notna()]
print(gsw_off_plays_df["seq_result_id"].isna().sum())
print(gsw_off_plays_df.shape[0])

0
22510


In [13]:
game_results_df.head()

,season,season_type,nba_game_id,home,away,game_date,quarter,poss_id,off_team,def_team,poss_time,poss_margin,poss_type,poss_press,poss_zone,oncourt_id,result_id,result_num,result_off,result_off_id,result_def1,result_def1_id,result_def2,result_def2_id,result_type,result_pbp_id,result_pbp_error,result_time,result_zone,result_contest,result_tov_reason,result_tov_teammate,result_foul_reason,isAND1,points,self_created,assisted,hcky_assisted,potential_ast,passer_name,passer_nba_id,hcky_passer_name,hcky_passer_nba_id,def1_play,def2_play,orb_win_name,orb_win_nba_id,drb_win_name,drb_win_nba_id,last_updated_time,did_score
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
267,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,3,LAL,GSW,694,0.0,Halfcourt,0,0,1,0022100002_1_3,1,LeBron James,2544.0,Stephen Curry,201939.0,NaN,NaN,TOV,11.0,NaN,687,Mid-Right Short Paint,NaN,Def Forced,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Russell Westbrook,201566.0,NaN,NaN,Steal,NaN,NaN,NaN,NaN,NaN,26:31.2,0
275,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,11,GSW,LAL,622,-2.0,Halfcourt,0,0,1,0022100002_1_11,1,Stephen Curry,201939.0,Russell Westbrook,201566.0,Kent Bazemore,203145.0,3+D Pull-Up,26.0,NaN,612,Mid-Right Above Break 3,Average,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26:31.2,0
283,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,17,GSW,LAL,532,-1.0,Halfcourt,0,0,1,0022100002_1_19,1,Kevon Looney,1626172.0,DeAndre Jordan,201599.0,NaN,NaN,Off-Lob / Tip,45.0,NaN,524,Rim,Open,NaN,NaN,NaN,0.0,2.0,0.0,1.0,1.0,1.0,Draymond Green,203110.0,Jordan Poole,1629673.0,NaN,NaN,NaN,NaN,NaN,NaN,26:31.2,1
291,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,24,GSW,LAL,446,0.0,Halfcourt,0,0,1,0022100002_1_26,1,Draymond Green,203110.0,Kent Bazemore,203145.0,NaN,NaN,TOV,61.0,NaN,442,Mid-Left Midrange,NaN,Off Fault,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26:31.2,0
299,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,32,LAL,GSW,374,2.0,Halfcourt,0,0,3,0022100002_1_34,1,LeBron James,2544.0,Andre Iguodala,2738.0,NaN,NaN,Off-Movement,82.0,NaN,370,Left Above Break 3,Average,NaN,NaN,NaN,0.0,3.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26:31.2,1


In [14]:
# Create master table of all games, joined on result id
game_data_master = gsw_off_plays_df.merge(game_results_df, how = "left", left_on = "seq_result_id", right_on = "result_id", suffixes = ('', "_y"))
cols_to_drop = [x for x in game_data_master.columns if x.endswith('_y')]
game_data_master.drop(cols_to_drop, axis = 1, inplace = True)
game_data_master.drop(axis = 1, columns = ["off_def", "last_updated_time"], inplace = True)
game_data_master.head()

,season,season_type,nba_game_id,home,away,game_date,quarter,poss_id,off_team,def_team,poss_time,oncourt_id,play_type,player_type,player_name,player_nba_id,play_clock,sequence_id,play_id,result_id,seq_result_id,play_zone,poss_margin,poss_type,poss_press,poss_zone,result_num,result_off,result_off_id,result_def1,result_def1_id,result_def2,result_def2_id,result_type,result_pbp_id,result_pbp_error,result_time,result_zone,result_contest,result_tov_reason,result_tov_teammate,result_foul_reason,isAND1,points,self_created,assisted,hcky_assisted,potential_ast,passer_name,passer_nba_id,hcky_passer_name,hcky_passer_nba_id,def1_play,def2_play,orb_win_name,orb_win_nba_id,drb_win_name,drb_win_nba_id,did_score
0,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,22,GSW,LAL,471,1,Closeout,Ball-Handler,Andrew Wiggins,203952.0,464,26,50,0022100002_1_24,0022100002_1_24,Right Above Break 3,-3.0,Halfcourt,0,0,1,Andrew Wiggins,203952.0,DeAndre Jordan,201599.0,NaN,NaN,Face-Up,58.0,NaN,462,Right Corner 3,Average,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,Stephen Curry,201939.0,Kevon Looney,1626172.0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,29,GSW,LAL,396,2,Ball Screen,Ball-Handler,Stephen Curry,201939.0,390,33,59,NaN,0022100002_1_31,Mid-Left Above Break 3,2.0,Halfcourt,0,0,1,Stephen Curry,201939.0,Anthony Davis,203076.0,NaN,NaN,Layup,75.0,NaN,380,Rim,Block,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Draymond Green,203110.0,Stephen Curry,201939.0,NaN,NaN,NaN,NaN,Anthony Davis,203076.0,0
2,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,39,GSW,LAL,301,4,Off-Move,Ball-Handler,Stephen Curry,201939.0,297,44,80,NaN,0022100002_1_41,Mid-Left Above Break 3,6.0,Halfcourt,0,0,1,Andre Iguodala,2738.0,LeBron James,2544.0,NaN,NaN,Floater,98.0,NaN,294,Rim,Average,NaN,NaN,NaN,0.0,2.0,0.0,1.0,0.0,1.0,Stephen Curry,201939.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,2,GSW,LAL,711,1,Off-Cut,Ball-Handler,Kevon Looney,1626172.0,702,2,5,0022100002_1_2,0022100002_1_2,Right Midrange,2.0,Halfcourt,0,0,1,Kevon Looney,1626172.0,Russell Westbrook,201566.0,NaN,NaN,Layup,9.0,NaN,700,Rim,Average,NaN,NaN,NaN,0.0,2.0,0.0,1.0,1.0,1.0,Stephen Curry,201939.0,Draymond Green,203110.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,5,GSW,LAL,683,1,Iso,Ball-Handler,Jordan Poole,1629673.0,681,5,9,0022100002_1_5,0022100002_1_5,Mid-Right Midrange,0.0,Halfcourt,0,0,1,Jordan Poole,1629673.0,Russell Westbrook,201566.0,NaN,NaN,1-2D Pull-Up,15.0,NaN,681,Mid-Right Midrange,Average,NaN,NaN,NaN,0.0,2.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [15]:
game_data_master

,season,season_type,nba_game_id,home,away,game_date,quarter,poss_id,off_team,def_team,poss_time,oncourt_id,play_type,player_type,player_name,player_nba_id,play_clock,sequence_id,play_id,result_id,seq_result_id,play_zone,poss_margin,poss_type,poss_press,poss_zone,result_num,result_off,result_off_id,result_def1,result_def1_id,result_def2,result_def2_id,result_type,result_pbp_id,result_pbp_error,result_time,result_zone,result_contest,result_tov_reason,result_tov_teammate,result_foul_reason,isAND1,points,self_created,assisted,hcky_assisted,potential_ast,passer_name,passer_nba_id,hcky_passer_name,hcky_passer_nba_id,def1_play,def2_play,orb_win_name,orb_win_nba_id,drb_win_name,drb_win_nba_id,did_score
0,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,22,GSW,LAL,471,1,Closeout,Ball-Handler,Andrew Wiggins,203952.0,464,26,50,0022100002_1_24,0022100002_1_24,Right Above Break 3,-3.0,Halfcourt,0,0,1,Andrew Wiggins,203952.0,DeAndre Jordan,201599.0,NaN,NaN,Face-Up,58.0,NaN,462,Right Corner 3,Average,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,1.0,Stephen Curry,201939.0,Kevon Looney,1626172.0,NaN,NaN,NaN,NaN,NaN,NaN,0
1,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,29,GSW,LAL,396,2,Ball Screen,Ball-Handler,Stephen Curry,201939.0,390,33,59,NaN,0022100002_1_31,Mid-Left Above Break 3,2.0,Halfcourt,0,0,1,Stephen Curry,201939.0,Anthony Davis,203076.0,NaN,NaN,Layup,75.0,NaN,380,Rim,Block,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Draymond Green,203110.0,Stephen Curry,201939.0,NaN,NaN,NaN,NaN,Anthony Davis,203076.0,0
2,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,39,GSW,LAL,301,4,Off-Move,Ball-Handler,Stephen Curry,201939.0,297,44,80,NaN,0022100002_1_41,Mid-Left Above Break 3,6.0,Halfcourt,0,0,1,Andre Iguodala,2738.0,LeBron James,2544.0,NaN,NaN,Floater,98.0,NaN,294,Rim,Average,NaN,NaN,NaN,0.0,2.0,0.0,1.0,0.0,1.0,Stephen Curry,201939.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,2,GSW,LAL,711,1,Off-Cut,Ball-Handler,Kevon Looney,1626172.0,702,2,5,0022100002_1_2,0022100002_1_2,Right Midrange,2.0,Halfcourt,0,0,1,Kevon Looney,1626172.0,Russell Westbrook,201566.0,NaN,NaN,Layup,9.0,NaN,700,Rim,Average,NaN,NaN,NaN,0.0,2.0,0.0,1.0,1.0,1.0,Stephen Curry,201939.0,Draymond Green,203110.0,NaN,NaN,NaN,NaN,NaN,NaN,1
4,2021,Regular Season,22100002,LAL,GSW,10/19/2021,1,5,GSW,LAL,683,1,Iso,Ball-Handler,Jordan Poole,1629673.0,681,5,9,0022100002_1_5,0022100002_1_5,Mid-Right Midrange,0.0,Halfcourt,0,0,1,Jordan Poole,1629673.0,Russell Westbrook,201566.0,NaN,NaN,1-2D Pull-Up,15.0,NaN,681,Mid-Right Midrange,Average,NaN,NaN,NaN,0.0,2.0,1.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22506,2021,Playoffs,42100406,BOS,GSW,6/16/2022,4,42,GSW,BOS,112,32,Perim Gather,Ball-Handler,Andrew Wiggins,203952.0,106,59,101,0042100406_4_47,0042100406_4_47,Right Corner 3,-12.0,Halfcourt,0,0,1,Andrew Wiggins,203952.0,NaN,NaN,NaN,NaN,Catch & Shoot,630.0,NaN,105,Right Corner 3,Open,NaN,NaN,NaN,0.0,3.0,0.0,1.0,0.0,1.0,Stephen Curry,201939.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
22507,2021,Playoffs,42100406,BOS,GSW,6/16/2022,4,46,GSW,BOS,78,32,Backcourt,Ball-Handler,Stephen Curry,201939.0,78,65,109,NaN,0042100406_4_52,Deep Backcourt,-13.0,Secondary,0,0,1,Klay Thompson,202691.0,NaN,NaN,NaN,NaN,TOV,641.0,NaN,73,Mid-Right Short Paint,NaN,Off Fault,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Stephen Curry,201939.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
22508,2021,Playoffs,42100406,BOS,GSW,6/16/2022,4,46,GSW,BOS,78,32,Off-Move,Ball-Handler,Klay Thompson,202691.0,74,65,110,0042100406_4_52,0042100406_4_52,Right Above Break 3,-13.0,Secondary,0,0,1,Klay Thompson,202691.0,NaN,NaN,NaN,NaN,TOV,641.0,NaN,73,Mid-Right Short Paint,NaN,Off Fault,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,Stephen Curry,201939.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
22509,2021,Playoffs,

In [16]:
#targets = ['result_type', 'result_zone', 'result_contest']
targets = ['result_type']
attributes = ["quarter", 
             "play_type", "player_name", "play_zone", "poss_margin", 
             "poss_type", "poss_press", "poss_zone"]

X = pd.get_dummies(game_data_master[attributes])
y = pd.get_dummies(game_data_master[targets])

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.30, random_state = 0)


X_train.shape, X_test.shape, y_train.shape, y_test.shape

((15757, 67), (6754, 67), (15757, 20), (6754, 20))

In [18]:
y.to_numpy().reshape(-1)

array([0, 0, 0, ..., 0, 0, 0], dtype=uint8)

In [19]:
from sklearn.metrics import mean_squared_error, f1_score
import xgboost as xgb

In [21]:
from sklearn.metrics import mean_squared_error, f1_score
from xgboost import cv

n_estimators = [2,3,4,5,10,20,30,40,50,100]
boosters = ["gbtree", "gblinear", "dart"]
alphas = [0, 0.1, 0.3, 0.5, 0.7, 0.9]
grow_policies = ['depthwise']
max_leaves = [0]
# n_jobs = -1

train_errors = []
test_errors = []


for n in n_estimators:
    for b in boosters:
        for a in alphas:
            for g in grow_policies:
                for l in max_leaves:
                        xgb_regressor = xgb.XGBClassifier(n_estimators = n,
                                                          booster = b,
                                                          n_jobs = -1,
                                                          alpha = a, 
                                                          random_state = 0,
                                                          grow_policy = g,
                                                          max_leaves = l,
                                                          missing = np.nan,
                                                          eval_metric = f1_score)
                                                         
                                                         '''
                                                         ,
                                                          tree_method='gpu_hist',
                                                          gpu_id=0
                                                          )
                                                          
                                                          # Mikey ADDED GPU compatibility to test improved runtimes in the event we have to rerun.
                                                          # https://xgboost.readthedocs.io/en/stable/gpu/index.html
                                                         '''
                                                        
                        
                        xgb_regressor.fit(X_train, y_train)
                        xgb_regressor_test_pred = xgb_regressor.predict(X_test)
                        xgb_regressor_train_pred = xgb_regressor.predict(X_train)
                        train_f1 = f1_score(xgb_regressor_train_pred, y_train, average = "weighted")
                        test_f1 = mean_squared_error(xgb_regressor_test_pred, y_test, average = "weighted")
                        train_errors.append([n,b,a,g,l,train_f1])
                        print([n,b,a,g,l,test_mse])
                        test_errors.append([n,b,a,g,l,test_f1])


test_error_opt = [e for e in test_errors if e[-1] == min(f[-1] for f in test_errors)]
print(test_error_opt)

                        
                                                        


IndentationError: unexpected indent (3008339706.py, line 30)

,quarter,poss_margin,poss_press,poss_zone,home_ATL,home_BKN,home_BOS,home_CHA,home_CHI,home_CLE,home_DAL,home_DEN,home_DET,home_GSW,home_HOU,home_IND,home_LAC,home_LAL,home_MEM,home_MIA,home_MIL,home_MIN,home_NOP,home_NYK,home_OKC,home_ORL,home_PHI,home_PHX,home_POR,home_SAC,home_SAS,home_TOR,home_UTA,home_WAS,away_ATL,away_BKN,away_BOS,away_CHA,away_CHI,away_CLE,away_DAL,away_DEN,away_DET,away_GSW,away_HOU,away_IND,away_LAC,away_LAL,away_MEM,away_MIA,away_MIL,away_MIN,away_NOP,away_NYK,away_OKC,away_ORL,away_PHI,away_PHX,away_POR,away_SAC,away_SAS,away_TOR,away_UTA,away_WAS,off_team_GSW,def_team_ATL,def_team_BKN,def_team_BOS,def_team_CHA,def_team_CHI,def_team_CLE,def_team_DAL,def_team_DEN,def_team_DET,def_team_HOU,def_team_IND,def_team_LAC,def_team_LAL,def_team_MEM,def_team_MIA,def_team_MIL,def_team_MIN,def_team_NOP,def_team_NYK,def_team_OKC,def_team_ORL,def_team_PHI,def_team_PHX,def_team_POR,def_team_SAC,def_team_SAS,def_team_TOR,def_team_UTA,def_team_WAS,play_type_Backcourt,play_type_Ball Screen,play_type_Closeout,play_type_DHO,play_type_Fake DHO,play_type_Inbounds,play_type_Initiation,play_type_Iso,play_type_Mid Gather,play_type_Off-Cut,play_type_Off-Move,play_type_Paint Gather,play_type_Perim Gather,play_type_Post,play_type_Rim Gather,play_type_Short Roll,play_type_Transition,player_name_Andre Iguodala,player_name_Andrew Wiggins,player_name_Chris Chiozza,player_name_Damion Lee,player_name_Draymond Green,player_name_Gary Payton II,player_name_Jeff Dowtin,player_name_Jonathan Kuminga,player_name_Jordan Poole,player_name_Juan Toscano-Anderson,player_name_Kevon Looney,player_name_Klay Thompson,player_name_Moses Moody,player_name_Nemanja Bjelica,player_name_Otto Porter Jr.,player_name_Quinndary Weatherspoon,player_name_Stephen Curry,play_zone_Deep Backcourt,play_zone_Left Above Break 3,play_zone_Left Corner 3,play_zone_Left Midrange,play_zone_Left Short Mid,play_zone_Mid-Left Above Break 3,play_zone_Mid-Left Deep Paint,play_zone_Mid-Left Midrange,play_zone_Mid-Left Short Paint,play_zone_Mid-Right Above Break 3,play_zone_Mid-Right Deep Paint,play_zone_Mid-Right Midrange,play_zone_Mid-Right Short Paint,play_zone_Right Above Break 3,play_zone_Right Corner 3,play_zone_Right Midrange,play_zone_Right Short Mid,play_zone_Rim,play_zone_Short Backcourt,poss_type_B/S OB,poss_type_EOQ Foul,poss_type_Early TO,poss_type_Halfcourt,poss_type_No Chance,poss_type_Off-FT REB,poss_type_Other,poss_type_Secondary,poss_type_Short EOQ,poss_type_Transition
690,1,0.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
10823,2,1.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0
19094,1,3.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
1061,2,11.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
7163,1,-4.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0

In [20]:
# optimal params: [100, 'gbtree', 0.9, 'depthwise', 0, 0.037781075]

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from xgboost import cv
import xgboost as xgb

'''
oracle = xgb.XGBClassifier(n_estimators = 3, booster = 'gbtree', 
                           n_jobs = -1, alpha = 0, random_state = 0, 
                           grow_policy = 'depthwise', max_leaves = 0, 
                           missing = np.nan, eval_metric = f1_score,
                           use_label_encoder= False)
'''

oracle = Pipeline([('minmax', MinMaxScaler()), ('xgb', xgb.XGBClassifier(n_estimators = 100, booster = 'gbtree', 
                           n_jobs = -1, alpha = 0.9, random_state = 0, 
                           grow_policy = 'depthwise', max_leaves = 0, 
                           missing = np.nan, eval_metric = f1_score))])

oracle.fit(X_train, y_train)

C:\Users\perso\anaconda3\envs\Shark_env\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


ValueError: y should be a 1d array, got an array of shape (15757, 20) instead.

In [ ]:
test_pred = oracle.predict(X_test)
train_pred = oracle.predict(X_train)



test_f1 = f1_score(y_test, test_pred, average = 'weighted')
train_f1 = f1_score(y_train, train_pred, average = 'weighted')

In [ ]:
print(test_f1, train_f1)

In [ ]:
type(y_test)
type(test_pred)

In [ ]:
from sklearn.metrics import confusion_matrix
cd = confusion_matrix(test_pred.argmax(axis=1), y_test.values.argmax(axis=1))

sns.heatmap(cd, annot=False, linewidth=.1, cmap=sns.cubehelix_palette(as_cmap=True))

In [43]:
current_time = '2020-07-26 02:12:18:'
print(current_time)

2020-07-26 02:12:18:
